In [3]:
# Author : Baptiste PICARD
# email : picard.baptiste@laposte.net
# First implementation :26/05/2019
# Last Changes : 18/06/2019
# Version : V0.4 - Setting model to exercise's consign and documentation
# For : IA2 UNcuyo project 
# Abstract : Short implementation of an MNIST classification (Handwritten's letters).
# Links to pages i used : 
#    - https://github.com/trekhleb/homemade-machine-learning/tree/master/homemade/neural_network
#    - https://www.youtube.com/watch?v=W8AeOXa_FqU&app=desktop
#    - https://www.youtube.com/watch?v=Dso6nQNGrrw&list=PLjyc6gCk4VU3whCnowAFbVfiBpztCpOKL&index=11
#    - https://towardsdatascience.com/how-does-back-propagation-in-artificial-neural-networks-work-c7cad873ea7

# Import for the project 
import time
import csv
import random
from math import exp, tanh
import numpy as np
import scipy.misc as smp
from threading import Thread, RLock

# import for the test, with Keras, TensorFlow
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

datas = 'A_Z Handwritten Data.csv'

In [2]:
# This function is used to draw an image of an Handwritten letter.
# It will transform a list of 785 element (which is an int between 0 = black and 255 = white)
# Into a numpy.array() used to draw the handwritten letter.
# Input = list python of 285 element.
# Output = np.array(28 * 28), image type (use to show)
def drawImage(numbers) :
    image = smp.toimage(numbers)
    image.show()
    return image
    

# This function will prepare the datas for the drawing and the learning. 
# Input : <Dataframe> which contains all the datas.
# Output[1] : Content of the .csv's file shaped to show images 785 -> (1, 28, 28). 
# Output[2] : Content of the .csv's file shaped to be an entry of the MLP : Dense. 
# Output[3] : Content of the .csv's file shaped to be an entry of the MLP : Conv2D. 
def shape_datas(datas) :
    start = time.time()
    with open(datas, 'r') as file : 
        datas = csv.reader(file)
        results = np.array(np.zeros(784).reshape(1, 28, 28))
        entries = np.array(np.zeros(784).reshape(1, 784))
        outputs = np.array(np.zeros(26).reshape(1, 26))
        limit = 30
        state = 0 
        for data in datas :
            if(state == limit) :
                break 
            else : 
                state += 1
            new_row = np.zeros(784)
            new_output = np.zeros(26)
            for index_element, element in enumerate(data[1:]) :
                new_row[index_element] = float(int(element)/255.0)
            new_output[int(new_row[:1])] = 1.0
            new_result = new_row.reshape(1, 28, 28)
            new_entry = new_row.reshape(1, 784)
            new_output = new_output.reshape(1, 26)
            results = np.append(results, new_result, axis=0)
            entries = np.append(entries, new_entry, axis=0)
            outputs = np.append(outputs, new_output, axis=0)
    print('It takes : ',(time.time()-start)/60,'.')
    return results, entries, outputs  
    
# This function is used to split the datas into a train's datas and test's datas.
# Ratio : 0,75 train, 0,25 test.
# Into a numpy.array() used to draw the handwritten letter.
# Input = np.array -> inputs and outputs, from the data's csv.
# Output = X_train, X_test, Y_train, Y_test.
def split_datas(inputs, outputs) :
    return train_test_split(inputs, outputs, test_size=0.25)

# Create a mlp with keras-tensorflow - 2 capas ocultas, to resolve a classification's problem
def getModel_Dense(num_pixels, num_classes):
    model = Sequential()
    model.add(Dense(512, input_shape=(num_pixels,), activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model 

# Create a convolutionnal model, to resolve a classification's problem
def getModel_CV(num_pixels, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(num_pixels, num_pixels, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model 


#image_shaped, inputs_shaped, outputs_shaped = openCSV_and_shape_datas(datas, 2000)
#print("X_train[0] dense's model is size  : ",X_train[0].size, "and type : ",type(X_train[0]))
#print("X_train[0] conv2D's model is size  : ",X_t[0].size, "and type : ",type(X_t[0]))
#print("Y_train[0] is size  : ",Y_train[0].size, "and type : ",type(X_train[0]))
#print("-----------------------------------------------------------------------")
#print("Type of X_train for dense's model : ",type(X_train))
#print(" and shape : ",X_train.shape, " and shape of an element : ",X_train[0].shape)
#print("Type of X_train for Conv2d's model : ",type(X_t))
#print(" and shape : ",X_t.shape, " and shape of an element : ",X_t[0].shape)
#print("-----------------------------------------------------------------------")
#print("Type of Y : ",type(Y_train))
#print(" and shape : ",Y_train.shape," and shape of an element : ",Y_train[0].shape)

input_results, input_entries, outputs = shape_datas(datas)
X_train, X_test, Y_train, Y_test = split_datas(input_results, outputs)
X_t, X_te, Y_t, Y_te = split_datas(input_entries, outputs)

It takes :  0.0009827931722005208 .


In [68]:
# Creating the activation function : sigmoid
def activation_sigmoid(x) :
    return 1 /(1 + exp(-x))

# Derivating the activation function : sigmoid
def activation_sigmoid_derivated(x) :
    #derivada: e^(-x) / (1 + e^(-x))^2
    return exp(-x) /(1 + exp(-x))**2

# Creating the activation function : tanh
def activation_tanh(x) :
    return tanh(x)

# Derivating the activation function : tanh
def activation_tanh_derivated(x) :
    return 1-tanh(x)

# Creating the activation function : tangent
def activation_tangent(x) : 
    return tan(x)

# Derivating the activation function : tangent
def activation_tangent_derivated(x) :
    return 1 + tan(x)**2

# Creating the activation function : softmax
def activation_softmax(x) : 
    return x

# Derivating the activation function : softmax
def activation_softmax_derivated(x) :
    return 1
    

# class MLP : multi-layer perceptron to learn about datas
# Create this class with several parameters
class MultilayerPerceptron :
    
    # Init this class with information.
    def __init__(self, name, datas, layers, learning_rate=0.30, activation=['tanh', 'tanh', 'tanh']) : 
        self.name = name # Name of the MLP, String 
        self.datas = datas # Datas to train the model, np.array([X_train, Y_train, X_test, Y_test])
        self.layers = layers # Define the number of layers of the MLP and the number of neurons on each layer, np.array([784, 512, 26])
        self.learning_rate = learning_rate # Learning rate : 0.30
        self.activation = activation # Define the activation's function for each layer : np.array(['sigmoid', 'sigmoid', 'softmax'])
        self.weights = self.create_model_weights() # Neuron's weights of the model 
        self.values = self.create_model_values() # Neuron's values of the model 
        self.new_values = self.create_model_values() # Copy of self.values, use to back_propagated the error
        self.old_weights = self.create_model_weights() # Copy of self.weights, use to back-propagate the error 
    
    # Getter and Setter of the variables of the model.
    # ------------------------------------------------
    def get_name(self) : # Return the name of the MLP
        return self.name 
    
    def get_layers(self) : # Return a list of the 3 layers and the number of neurons of each one 
        return self.layers
    
    def get_layer_index(self, index) : # Return an integer corresponding to the number of neurons of the layer : index
        return self.layers[index]
    
    def get_X_train(self) : # Return X_train
        return self.datas[0]
    
    def get_Y_train(self) : # Return Y_train 
        return self.datas[1]
    
    def get_X_test(self) : # Return X_test
        return self.datas[2] 
    
    def get_Y_test(self) : # Return Y_test
        return self.datas[3]
    
    def get_learning_rate(self) : # Return the learning rate 
        return self.learning_rate
    
    def set_learning_rate(self, new_learning_rate) : # Set the learning rate to a new value 
        if(new_learning_rate>=0.0 and new_learning_rate<=1.0) :
            self.learning_rate = new_learning_rate
    
    def get_activations(self) : # Return the list with the correspondings activation functions of each layer of MLP.
        return self.activation
    
    def get_activation_layer(self, layer) : # Return the activation function (String) corresponding to the layer : layer 
        return self.activation[layer]
    
        # For the variable : weights 
    # -----------------------
    def get_model_weights(self) : # Return all the weights (np.array) of the model.
        return self.weights
    
    def get_model_weights_layer(self, layer, index_value_associated) : # Return a layer of the weights associated to a specific value 
        return self.weights[layer][index_value_associated]
    
    def get_weight_neuron(self, x, y, z) : # Get a weight of a neuron in self.weights
        return self.weights[x][y][z]
    
    def set_weight_neuron(self, x, y, z, value) : # Set the weight of a neuron -> in self.weghts
        if(value >= 0.0 and value <= 1.0) :
            self.weights[x][y][z] = value
            
    def set_model_weights_layer(self, layer, index, new_layer) : # Set the weights associated to a value (np.array())
        self.weights[layer][index] = new_layer
    
        # For the variable : values 
    # -----------------------
    def get_model_values(self) : # Return all the values (np.array) of the model
        return self.values
    
    def get_model_values_layer(self, layer) : # Return a layer of the values
        return self.values[layer]
            
    def set_model_values_layer(self, layer, new_layer) : # Set a layer of values (np.array())
        self.values[layer] = new_layer
    
    def get_value_neuron(self, x, y) : # Get a value of a neuron in self.values
        return self.values[x][y]
    
    def set_value_neuron(self, x, y, value) : # Set the value of a neuron -> in self.values
        if(value >= 0.0 and value <= 1.0) :
            self.values[x][y] = value
    
        # For the variable : new_values 
    # -----------------------
    def get_model_new_values(self) : # Return all the new_values (np.array) of the model -> used for the back-propagation
        return self.new_values

    def get_new_value_neuron(self, x, y) : # Get a value of a neuron in self.new_values
        return self.new_values[x][y]
    
    def set_new_values_neuron(self, x, y, value) : # Set an item of new_values to value
        self.new_values[x][y] = value 
        
        # For the variable : old_weights 
    # -----------------------
    def get_model_old_weights(self) : # Return all the old_weights (np.array) of the model -> used for the back-propagation
        return self.old_weights
    
    def get_old_weight_neuron(self, x, y, z) : # Get a the old_weight of a neuron in self.old_weights
        return self.old_weights[x][y][z]
    
    def set_old_weights_neuron(self, x, y, z, value) : # Set an item of new_values to value
        self.old_weights[x][y][z] = value 
        
        # Functions to initialise the weights and values.
    # ------------------------------------------------
    def create_model_weights(self) : # Create the weights of the model, each weight is initialise between [-1.0 : 1.0]
        init_model = []
        for index_layer,layer in enumerate(self.get_layers()[:-1]) :
            new_weights = []
            for i in range(self.get_layer_index(index_layer+1)) :
                new = np.zeros(layer)
                for index in range (new.size) :
                    new[index] = random.uniform(-1.0, 1.0)
                new_weights.append(new)
            init_model.append(np.array(new_weights))
        return np.array(init_model)
            
    def create_model_values(self) : # Create the values of the model, each value is initialise between [0.0 : 1.0]
        init_model = []
        for index, layer in enumerate(self.get_layers()) : 
            if(index==0) :
                init_model.append(np.zeros(layer))
            else : 
                new_layer=np.zeros(layer)
                for i in range(layer) :
                    new_layer[i] = random.random()
                init_model.append(new_layer) 
        return np.array(init_model)
        
    def copy_values_to_new_values(self) : # Copy all the float of values into new_values
        for index_layer, layer in enumerate(self.get_model_values()) :
            for index_value, value in enumerate(layer) :
                self.set_new_values_neuron(index_layer, index_value, value) 
                
    def copy_weights_to_old_weights(self) : # Copy all the float of weights into weights
        for index_layer, layer in enumerate(self.get_model_weights()) :
            for index_value, value in enumerate(layer) :
                for index_item, item in enumerate(value) :
                    self.set_old_weights_neuron(index_layer, index_value, index_item, item) 
        
        # Functions to activate a float with a function or its derivated.
    # ------------------------------------------------        
    def activate(self, function, value) :
        # Using a value (float) and a function (String), return the result of the value in the function
        if(function == 'sigmoid' ) :
            return activation_sigmoid(value)
        elif(function == 'tangent' ) :
            return activation_tangent(value)
        elif(function == "softmax") :
            return activation_softmax(value)
        elif(function=='tanh') :
            return activation_tanh(value)
        
    def activate_derivated(self, function, value) : 
        # Using a value (float) and a function (String), return the result of the value in the derivated's function
        if(function == 'sigmoid' ) :
            return activation_sigmoid_derivated(value)
        elif(function == 'tangent' ) :
            return activation_tangent_derivated(value)
        elif(function == "softmax") :
            return activation_softmax_derivated(value)
        elif(function == 'tanh') :
            return activation_tanh_derivated(value)
        
        # Functions to propagate the value, backpropagate the error or to show the model.
    # ------------------------------------------------     
    def show_model(self) : # Show the model -> used to make a summary of the MLP
        print("--------------------- Explaining the model ------------------------")
        print("This model is composed by", self.get_model_weights().size, "simples layers.\n")
        for index, layer in enumerate(self.get_layers()) :
            print('The',index+1,"s layer got :",layer,"neurons.")
            print("The activation's function associated is : ",self.activation[index],'.\n')
        print("-------------------------------------------------------------------")
    
    def test_model_and_show_error(self) : # Use to test and recuperate different errors
        pass
    
    def propagation(self, X) : 
        # Using an entry (of X_train) -> We propagate the entry to the output layer.
        # To propagate the entry to the output, we use the weights of a value and the values of the layers.
        # The next_value = activation(sum(value_i * weight_i) - actual_value)
        # test size X_train -> weights.size at inputs
        if(X.size == self.get_model_values()[0].size) : # Test the entry size : X_train <-> model
            # For each layer of the MLP -> 3 layers : 0, 1, 2
            for index_layer in range(self.get_layers().size) : 
                
                # The first layer correspond to the values o X_train.
                # We simply copy the entry'values into values
                if(index_layer == 0) : 
                    self.set_model_values_layer(0, X) # Set the value's layers with X_train's values
                    self.copy_values_to_new_values() # Copy actual values to new values
                    
                # Others layers need propagation using first layer.
                # Using the first layer (train values), we propagate the values to the next layer using the weights,
                # The old values, and the activation's functions.
                # This is the first step, which helps to predict the output using various entries.
                # With the propagation, we can see if we need to set the weight between layers and the values of the neurons 
                # Or if we can just keep or model.
                else : 
                    # For each value in the next layer [1, 2] -> the layer 0 was call before
                    for index_value, value in enumerate(self.get_model_values_layer(index_layer)) :
                        new_val = 0.0 # New value by propagation  
                        # Propagation's equation we will use after : 
                        # To calculate the next value of a neuron with the neurons of the previous layer, 
                        # new_val = activation_function( sum(previous_neuron_i * weight_previous_to_next_neuron) - actual_value ).
                        for index_previous_value, previous_value in enumerate(self.get_model_values_layer(index_layer-1)) : 
                            new_val = new_val + previous_value * self.get_weight_neuron(index_layer-1, index_value, index_previous_value) 
                        new_val = self.activate(self.get_activation_layer(index_layer), new_val - self.get_value_neuron(index_layer, index_value)) # Activating with the right function
                        self.set_value_neuron(index_layer, index_value, new_val) # Set the value to the value propagated
        else : 
            print('Error in shaping entries.')
            print('X : ',X.size,' and : ',self.get_model_values_layer(0).size)
            
        
    def backpropagation(self, out_layer):
        delta_d = np.zeros(self.get_model_values_layer(2).size) # creating the deltas, using to give the new value/weight of a layer/neuron.    
        self.copy_weights_to_old_weights() # Creating a copy of the weigths.
        if(out_layer.size == self.get_model_values_layer(2).size) : # test size X_train -> weights.size at inputs
            
            # For each layer of the MLP (3 layer) -> Layers : 2, 1, 0
            for index_layer in range(self.get_layers().size-1,0,-1) : 
                # Each neuron of the layer
                for index_value, value in enumerate(self.get_model_values_layer(index_layer)) :
                    
                    # We calculate the h_d, in the same way of the porpagation of a value, but we use the activation's function derivated
                    # To activate the value. The variable 'h_d' was made to propagate the error, we use it to each delta
                    # Of each layer.
                    h_ = 0.0 
                    for index_previous_value, previous_value in enumerate(self.get_model_values_layer(index_layer-1)) :
                        h_ += previous_value * self.get_weight_neuron(index_layer-1, index_value, index_previous_value)
                    h_ += - self.new_values[index_layer][index_value]
                    h_d = self.activate_derivated('tanh', h_) # h_d = activation_derivated(sum(w_i * v_i) - value_anterior)
                    
                    # If are of the last layer, we need to create the delta, using the difference between real and predicted output 
                    if (index_layer==2) : 
                        delta_d[index_value] = h_d*(out_layer[index_value] - self.get_value_neuron(index_layer, index_value)) # Creating the delta according to the neuron.
                        seters = np.zeros(self.get_model_values_layer(index_layer-1).size+1) # To save the variation of the weight/value's neuron
                        actual_w_v = np.zeros(self.get_model_values_layer(index_layer-1).size+1) # To get the weight/value's neuron
                        # For each item of the np.array(), we calculate the variation and save it 
                        for index_seter, seter in enumerate(seters) : 
                            # Corresponding to a value 
                            if(index_seter == 0) : 
                                seters[index_seter] = self.get_learning_rate() * delta_d[index_value] * out_layer[index_value] # Calculate the variation 
                                actual_w_v[index_seter] = self.get_new_value_neuron(index_layer, index_value) # Get the right value 
                            # Corresponding to weights
                            else : 
                                seters[index_seter] = self.get_learning_rate() * delta_d[index_value] * self.get_value_neuron(index_layer-1, index_seter-1) # Calculate the variation
                                actual_w_v[index_seter] = self.get_weight_neuron(index_layer-1, index_value, index_seter-1) # Get the right Weight
                        news = seters + actual_w_v # Get the new value and the news weights 
                    
                    # If we are not in the last layer, we don't calculate the delta with the same equation.
                    else : 
                        delta_d_1 = 0.0
                        # Caltulate the delta for each neuron of the hidden layer, we need the delta of the output layer
                        # delta = sum(output_neuron_delta * weight) * h_d
                        for index_delta, delta in enumerate(delta_d) :
                            delta_d_1 += delta * self.old_weights[index_layer][0][index_delta]
                        delta_d_1 = delta_d_1 * h_d # We calculate the new delta with the deltas of the output layer.
                        seters = np.zeros(self.get_model_values_layer(index_layer-1).size+1) # To save the variation of the weight/value's neuron
                        actual_w_v = np.zeros(self.get_model_values_layer(index_layer-1).size+1) # To get the weight/value's neuron
                        # For each item of the np.array(), we calculate the variation and save it
                        for index_seter, seter in enumerate(seters) : 
                            # Corresponding to a value
                            if(index_seter == 0) :
                                seters[index_seter] = self.get_learning_rate() * delta_d_1 * (-1) # Calculate the variation 
                                actual_w_v[index_seter] = self.get_new_value_neuron(index_layer, index_value) # Get the right value 
                            # Corresponding to weights
                            else :
                                seters[index_seter] = self.get_learning_rate() * delta_d_1 * self.get_value_neuron(index_layer-1, index_seter-1) # Calculate the variation
                                actual_w_v[index_seter] = self.get_weight_neuron(index_layer-1, index_value, index_seter-1) # Get the right Weight
                        news = seters + actual_w_v # Get the new value and the news weights      
                       
                    # When we finished to calculate the new values, we need to set the old value/weight by the new value/weight  
                    # For all items in 'new', we enulerate them
                    for index_new, new in enumerate(news) : 
                        # index = 0, it correspond to a value 
                        if(index_new == 0) :   
                            self.new_values[index_layer][index_value] = news[index_new] # We set the value of the right neuron by it new value
                        # index = not(0) , it correspond to a weight
                        else :   
                            self.set_weight_neuron(index_layer-1, index_value, index_new-1, news[index_new]) # We set the weight of the right neuron by it new weight
            
        else : 
            print('Error in shaping outputs.')
            print('Y : ',out_layer,' and : ',self.get_model_values_layer(2))
        
    def calculate_rendimiento(self) : 
        loss = 0.0
        accuracy = 0.0
        print("Loss : {0}, Accuracy : {1}".format(loss, accuracy))
        return loss, accuracy
            
X_t = X_t.reshape(23, 784)
mlp = MultilayerPerceptron("First_MLP", [X_t, Y_t, X_test, Y_test], np.array([784, 512, 26]), 0.3) # Creating a model, Multi Layer Perceptron
mlp.show_model() # Show informations
for i in range(0, len(mlp.get_X_train())-1) :
    print("Threat : {0} on {1}".format(i+1, len(mlp.get_X_train())-1))
    mlp.propagation(mlp.get_X_train()[i])
    mlp.backpropagation(mlp.get_Y_train()[i])
print("\n------------ threats terminated ------------")

--------------------- Explaining the model ------------------------
This model is composed by 2 simples layers.

The 1 s layer got : 784 neurons.
The activation's function associated is :  tanh .

The 2 s layer got : 512 neurons.
The activation's function associated is :  tanh .

The 3 s layer got : 26 neurons.
The activation's function associated is :  tanh .

-------------------------------------------------------------------
Threat : 1 on 22
Threat : 2 on 22
Threat : 3 on 22
Threat : 4 on 22
Threat : 5 on 22
Threat : 6 on 22
Threat : 7 on 22
Threat : 8 on 22
Threat : 9 on 22
Threat : 10 on 22
Threat : 11 on 22
Threat : 12 on 22
Threat : 13 on 22
Threat : 14 on 22
Threat : 15 on 22
Threat : 16 on 22
Threat : 17 on 22
Threat : 18 on 22
Threat : 19 on 22
Threat : 20 on 22
Threat : 21 on 22
Threat : 22 on 22

------------ threats terminated ------------


In [54]:
model_2d = getModel_CV(28, 26)
X_t = X_t.reshape(23, 28, 28, 1)
X_te = X_te.reshape(8, 28, 28, 1)
model_2d.fit(X_t, Y_t, validation_data=(X_te, Y_te), epochs=30, batch_size=50, verbose=2)
scores = model_2d.evaluate(X_te, Y_te, verbose=0)
print("Loss : ",int(scores[0]*100),"% / Accuracy : ",int(scores[1]*100),"%.")

Train on 23 samples, validate on 8 samples
Epoch 1/30
 - 2s - loss: 3.0910 - acc: 0.0435 - val_loss: 3.1475 - val_acc: 1.0000
Epoch 2/30
 - 0s - loss: 3.0294 - acc: 0.3478 - val_loss: 3.0692 - val_acc: 1.0000
Epoch 3/30
 - 0s - loss: 2.9617 - acc: 0.7391 - val_loss: 2.9718 - val_acc: 1.0000
Epoch 4/30
 - 0s - loss: 2.8717 - acc: 0.7391 - val_loss: 2.8513 - val_acc: 1.0000
Epoch 5/30
 - 0s - loss: 2.7732 - acc: 0.8261 - val_loss: 2.7012 - val_acc: 1.0000
Epoch 6/30
 - 0s - loss: 2.6221 - acc: 0.9130 - val_loss: 2.5176 - val_acc: 1.0000
Epoch 7/30
 - 0s - loss: 2.5103 - acc: 0.9565 - val_loss: 2.2942 - val_acc: 1.0000
Epoch 8/30
 - 0s - loss: 2.2872 - acc: 0.9565 - val_loss: 2.0237 - val_acc: 1.0000
Epoch 9/30
 - 0s - loss: 2.0849 - acc: 0.9565 - val_loss: 1.7019 - val_acc: 1.0000
Epoch 10/30
 - 0s - loss: 1.8307 - acc: 1.0000 - val_loss: 1.3336 - val_acc: 1.0000
Epoch 11/30
 - 0s - loss: 1.3579 - acc: 1.0000 - val_loss: 0.9438 - val_acc: 1.0000
Epoch 12/30
 - 0s - loss: 1.3007 - acc: 1.

In [51]:
model = getModel_Dense(784,26)
model.summary()
X_train = X_train.reshape(23, 784)
X_test = X_test.reshape(8, 784)
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=15, batch_size=50, verbose=2)
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Loss : ",int(scores[0]*100),"% / Accuracy : ",int(scores[1]*100),"%.")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_29 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_30 (Dense)             (None, 26)                13338     
Total params: 677,914
Trainable params: 677,914
Non-trainable params: 0
_________________________________________________________________
Train on 23 samples, validate on 8 samples
Epoch 1/15
 - 1s - loss: 3.2033 - acc: 0.0000e+00 - val_loss: 0.0190 - val_acc: 1.0000
Epoch 2/15
 - 0s - loss: 0.0698 - acc: 1.0000 - val_loss: 4.6803e-04 - val_acc: 1.0000
Epoch 3/15
 - 0s - loss: 0.0061 - acc: 1.0000 - val_loss: 1.9170e-04 - val_acc: 1.0000
Epoch 4/15
 - 0s - loss: 0.0034 - acc: 1.0000 - val_loss: 9.5674e-05 - val_acc: 1.0000
Epoch 5/1